In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
import pandas as pd
import pickle
from matplotlib.pyplot import MultipleLocator

In [ ]:
# 来源：中科院硕士论文-闻博
# 在自己的数据集上复现其模型

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
train_data = pd.read_csv(r'../input/blood-pressure-datasets/Train_Merge_Data.csv')
validation_data = pd.read_csv(r'../input/blood-pressure-datasets/Validation_Merge_Data.csv')

In [ ]:
train_data = shuffle(train_data)
validation_data = shuffle(validation_data)

In [ ]:
train_label_Sbp = train_data.iloc[:,610]
train_label_Sbp = train_label_Sbp.values

train_label_Dbp = train_data.iloc[:,611]
train_label_Dbp = train_label_Dbp.values

train_data = train_data.iloc[:,:600]
train_data = train_data.values

In [ ]:
validation_label_Sbp = validation_data.iloc[:,610]
validation_label_Sbp = validation_label_Sbp.values

validation_label_Dbp = validation_data.iloc[:,611]
validation_label_Dbp = validation_label_Dbp.values

validation_data = validation_data.iloc[:,:600]
validation_data = validation_data.values

In [ ]:
print("train_data_information:")
print(train_data.shape)
print(train_label_Sbp.shape)
print(train_label_Dbp.shape)
print("validation_data_information:")
print(validation_data.shape)
print(validation_label_Sbp.shape)
print(validation_label_Dbp.shape)

In [ ]:
x=32
def create_model():
    inputs = keras.Input(shape=(600,1))
    #1
    conv1 = layers.Conv1D(x*2,3,padding='same')(inputs)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Activation(tf.nn.relu)(conv1)
    
    conv1 = layers.Conv1D(x*2,3,padding='same')(conv1)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Activation(tf.nn.relu)(conv1)
    
    pool1 = layers.MaxPooling1D(strides=3)(conv1)
    #2
    conv2 = layers.Conv1D(x*4,3,padding='same')(pool1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Activation(tf.nn.relu)(conv2)

    conv2 = layers.Conv1D(x*4,3,padding='same')(conv2)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Activation(tf.nn.relu)(conv2)

    pool2 = layers.MaxPooling1D(strides=3)(conv2)
    
    #3
    conv3 = layers.Conv1D(x*8,3,padding='same')(pool2)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Activation(tf.nn.relu)(conv3)

    conv3 = layers.Conv1D(x*8,3,padding='same')(conv3)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Activation(tf.nn.relu)(conv3)
    
    conv3 = layers.Conv1D(x*8,3,padding='same')(conv3)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Activation(tf.nn.relu)(conv3)
    
    pool3 = layers.MaxPooling1D(strides=3)(conv3)
    
    #
    conv4 = layers.Conv1D(x*16,3,padding='same')(pool3)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Activation(tf.nn.relu)(conv4)

    conv4 = layers.Conv1D(x*16,3,padding='same')(conv4)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Activation(tf.nn.relu)(conv4)
    
    conv4 = layers.Conv1D(x*16,3,padding='same')(conv4)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Activation(tf.nn.relu)(conv4)
    
    pool4 = layers.MaxPooling1D(strides=3)(conv4)
    
    gru_bi = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(pool4)
    
    


    com_layer = layers.GlobalAveragePooling1D()(conv9)
    
    
    com_layer = layers.Dense(32,activation='relu')(com_layer)
    
    outputs_sbp = layers.Dense(1,name='Sbp')(com_layer)
    outputs_dbp = layers.Dense(1,name='Dbp')(com_layer)

    model = keras.Model(inputs=inputs,outputs=[outputs_sbp,outputs_dbp])
    
    return model

In [ ]:
x=32
inputs = keras.Input(shape=(600,1))
#1
conv1 = layers.Conv1D(x*2,3,padding='same')(inputs)
conv1 = layers.BatchNormalization()(conv1)
conv1 = layers.Activation(tf.nn.relu)(conv1)

conv1 = layers.Conv1D(x*2,3,padding='same')(conv1)
conv1 = layers.BatchNormalization()(conv1)
conv1 = layers.Activation(tf.nn.relu)(conv1)

pool1 = layers.MaxPooling1D(strides=3)(conv1)
#2
conv2 = layers.Conv1D(x*4,3,padding='same')(pool1)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Activation(tf.nn.relu)(conv2)

conv2 = layers.Conv1D(x*4,3,padding='same')(conv2)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Activation(tf.nn.relu)(conv2)

pool2 = layers.MaxPooling1D(strides=3)(conv2)

#3
conv3 = layers.Conv1D(x*8,3,padding='same')(pool2)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Activation(tf.nn.relu)(conv3)

conv3 = layers.Conv1D(x*8,3,padding='same')(conv3)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Activation(tf.nn.relu)(conv3)

conv3 = layers.Conv1D(x*8,3,padding='same')(conv3)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Activation(tf.nn.relu)(conv3)

pool3 = layers.MaxPooling1D(strides=3)(conv3)

#
conv4 = layers.Conv1D(x*16,3,padding='same')(pool3)
conv4 = layers.BatchNormalization()(conv4)
conv4 = layers.Activation(tf.nn.relu)(conv4)

conv4 = layers.Conv1D(x*16,3,padding='same')(conv4)
conv4 = layers.BatchNormalization()(conv4)
conv4 = layers.Activation(tf.nn.relu)(conv4)

conv4 = layers.Conv1D(x*16,3,padding='same')(conv4)
conv4 = layers.BatchNormalization()(conv4)
conv4 = layers.Activation(tf.nn.relu)(conv4)

pool4 = layers.MaxPooling1D(strides=3)(conv4)

gru_bi = layers.Bidirectional(layers.GRU(64,return_sequences=True))(pool4)

#attention

# batch_size, time_steps, lstm_units -> batch_size, lstm_units, time_steps
attention_layer = layers.Permute((2, 1))(gru_bi)
# batch_size, lstm_units, time_steps -> batch_size, lstm_units, time_steps
attention_layer = layers.Dense(7, activation='softmax')(attention_layer)
# batch_size, lstm_units, time_steps -> batch_size, time_steps, lstm_units
attention_layer_probs = layers.Permute((2, 1), name='attention_vec')(attention_layer)

# 相当于获得每一个step中，每个特征的权重
output_attention_mul = layers.multiply([gru_bi, a_probs], name='attention_mul', mode='mul')

output_attention_mul = layers.Flatten()(output_attention_mul)
print(output_attention_mul)

outputs_sbp = layers.Dense(1,name='Sbp')(output_attention_mul)
outputs_dbp = layers.Dense(1,name='Dbp')(output_attention_mul)

model = keras.Model(inputs=inputs,outputs=[outputs_sbp,outputs_dbp])

return model